In [1]:
# Import PySpark
from pyspark.sql import SparkSession
from pyspark.ml.feature import StandardScaler
from pyspark.ml.feature import VectorAssembler


In [2]:
# create a SparkSession object
spark = SparkSession.builder.appName('ImportCSV').getOrCreate()

# read the CSV file into a DataFrame
df = spark.read.options(header=True, inferSchema=True).csv('Data/creditcard.csv')

# show the DataFrame
df.show()

23/04/17 11:54:35 WARN Utils: Your hostname, server-gpu resolves to a loopback address: 127.0.1.1; using 192.168.99.222 instead (on interface enp9s0)
23/04/17 11:54:35 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/04/17 11:54:35 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/04/17 11:54:41 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+----+------------------+-------------------+------------------+-------------------+-------------------+-------------------+--------------------+-------------------+------------------+-------------------+------------------+------------------+-------------------+-------------------+-------------------+-------------------+--------------------+-------------------+-------------------+-------------------+--------------------+-------------------+-------------------+-------------------+-------------------+-------------------+--------------------+-------------------+------+-----+
|Time|                V1|                 V2|                V3|                 V4|                 V5|                 V6|                  V7|                 V8|                V9|                V10|               V11|               V12|                V13|                V14|                V15|                V16|                 V17|                V18|                V19|                V20|                 V

In [3]:
df.head(5)

[Row(Time=0.0, V1=-1.3598071336738, V2=-0.0727811733098497, V3=2.53634673796914, V4=1.37815522427443, V5=-0.338320769942518, V6=0.462387777762292, V7=0.239598554061257, V8=0.0986979012610507, V9=0.363786969611213, V10=0.0907941719789316, V11=-0.551599533260813, V12=-0.617800855762348, V13=-0.991389847235408, V14=-0.311169353699879, V15=1.46817697209427, V16=-0.470400525259478, V17=0.207971241929242, V18=0.0257905801985591, V19=0.403992960255733, V20=0.251412098239705, V21=-0.018306777944153, V22=0.277837575558899, V23=-0.110473910188767, V24=0.0669280749146731, V25=0.128539358273528, V26=-0.189114843888824, V27=0.133558376740387, V28=-0.0210530534538215, Amount=149.62, Class=0),
 Row(Time=0.0, V1=1.19185711131486, V2=0.26615071205963, V3=0.16648011335321, V4=0.448154078460911, V5=0.0600176492822243, V6=-0.0823608088155687, V7=-0.0788029833323113, V8=0.0851016549148104, V9=-0.255425128109186, V10=-0.166974414004614, V11=1.61272666105479, V12=1.06523531137287, V13=0.48909501589608, V14=-

In [4]:
# group the DataFrame by all columns
grouped_df = df.groupBy(df.columns).count()

# select rows where count is greater than 1
duplicates = grouped_df.filter('count > 1')

In [5]:
duplicates.show(), duplicates.count()

+-------+------------------+------------------+------------------+-------------------+-------------------+-------------------+-------------------+------------------+------------------+------------------+-------------------+------------------+-------------------+--------------------+-------------------+-------------------+-------------------+------------------+------------------+-------------------+-------------------+-------------------+--------------------+-------------------+-------------------+-------------------+-------------------+-------------------+------+-----+-----+
|   Time|                V1|                V2|                V3|                 V4|                 V5|                 V6|                 V7|                V8|                V9|               V10|                V11|               V12|                V13|                 V14|                V15|                V16|                V17|               V18|               V19|                V20|                V

(None, 773)

In [6]:
# remove duplicates
df = df.dropDuplicates()

In [7]:
# group the DataFrame by all columns
grouped_df = df.groupBy(df.columns).count()

# select rows where count is greater than 1
duplicates = grouped_df.filter('count > 1')

In [8]:
duplicates.show(), duplicates.count()


+----+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+------+-----+-----+
|Time| V1| V2| V3| V4| V5| V6| V7| V8| V9|V10|V11|V12|V13|V14|V15|V16|V17|V18|V19|V20|V21|V22|V23|V24|V25|V26|V27|V28|Amount|Class|count|
+----+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+------+-----+-----+
+----+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+------+-----+-----+



(None, 0)

In [9]:
# select the feature columns and the label column
label = df.select('Class')
feature = df.drop('Class')


In [10]:
feature.head(3), feature.columns

([Row(Time=26.0, V1=-0.529912284186556, V2=0.873891581460326, V3=1.34724732930113, V4=0.145456676582257, V5=0.414208858362661, V6=0.10022309405219, V7=0.711206082959649, V8=0.1760659570625, V9=-0.286716934699997, V10=-0.484687683196852, V11=0.872489590125871, V12=0.851635859904339, V13=-0.571745302934562, V14=0.100974273045751, V15=-1.51977183258512, V16=-0.284375978261788, V17=-0.310523584869201, V18=-0.404247868800905, V19=-0.823373523914155, V20=-0.290347610865436, V21=0.0469490671140629, V22=0.208104855076299, V23=-0.185548346773547, V24=0.00103065983293288, V25=0.0988157011025622, V26=-0.552903603040518, V27=-0.0732880835681738, V28=0.0233070451077205, Amount=6.14),
  Row(Time=158.0, V1=-0.600816388115364, V2=0.922454525911535, V3=-0.135951820418704, V4=-1.25991483910974, V5=2.43982440634166, V6=3.33020532398973, V7=0.0871070880575762, V8=0.949659242063477, V9=-0.327931944022326, V10=0.0617085198149349, V11=-0.157387072529051, V12=-0.233220336669169, V13=-0.0533324475993436, V14=0

In [11]:
label.head(10), label.columns

([Row(Class=0),
  Row(Class=0),
  Row(Class=0),
  Row(Class=0),
  Row(Class=0),
  Row(Class=0),
  Row(Class=0),
  Row(Class=0),
  Row(Class=0),
  Row(Class=0)],
 ['Class'])

I don't find a way to easily oversmapling and undersampling so use pandas

In [12]:
p_feature = feature.toPandas()
p_label = label.toPandas()

In [13]:
p_feature.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
0,26.0,-0.529912,0.873892,1.347247,0.145457,0.414209,0.100223,0.711206,0.176066,-0.286717,...,-0.290348,0.046949,0.208105,-0.185548,0.001031,0.098816,-0.552904,-0.073288,0.023307,6.14
1,158.0,-0.600816,0.922455,-0.135952,-1.259915,2.439824,3.330205,0.087107,0.949659,-0.327932,...,0.348028,-0.311489,-0.927233,-0.045268,0.979587,0.016423,0.091043,0.371280,0.180187,1.79
2,190.0,-1.505779,-0.215325,1.991294,-1.631493,-0.635965,0.228414,-0.034266,0.042696,-0.235440,...,0.232752,-0.101938,0.148146,-0.450115,-0.396558,0.554224,-0.340493,-0.335618,-0.413379,82.29
3,265.0,-0.491003,0.906953,1.645423,-0.083531,-0.195560,-0.710165,0.559119,0.116340,-0.538190,...,0.062757,-0.168067,-0.517387,0.018650,0.491652,-0.277795,0.043841,0.253372,0.111749,9.03
4,459.0,-0.528218,0.981232,1.652988,-0.150715,0.137048,-0.203980,0.583370,0.165246,-0.781696,...,0.127768,-0.123569,-0.272731,0.023887,0.223936,-0.297777,0.068300,0.281018,0.111557,6.99


In [14]:
p_label.head()

,Class
0,0
1,0
2,0
3,0
4,0


NearMiss and SMOTE

In [15]:
from imblearn.under_sampling import NearMiss
# Create an instance of NearMiss
nm = NearMiss()

# Fit and apply NearMiss to downsample the majority class
nm_features, nm_labels = nm.fit_resample(p_feature, p_label)

nm_features.shape, nm_labels.shape, nm_labels.value_counts()

((946, 30),
 (946, 1),
 Class
 0        473
 1        473
 dtype: int64)

In [16]:
from imblearn.pipeline import make_pipeline as make_pipeline_imb # To do our transformation in a unique time
from imblearn.over_sampling import SMOTE, ADASYN
from imblearn.metrics import classification_report_imbalanced

smote=SMOTE()
sm_features,sm_labels=smote.fit_resample(p_feature,p_label)
sm_features.shape,sm_labels.shape, sm_labels.value_counts()

((566506, 30),
 (566506, 1),
 Class
 0        283253
 1        283253
 dtype: int64)

Rejoin with feature and label

In [17]:
import pandas as pd

In [18]:
sm_dataset = pd.concat([sm_features, sm_labels], axis=1)
sm_dataset.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,26.0,-0.529912,0.873892,1.347247,0.145457,0.414209,0.100223,0.711206,0.176066,-0.286717,...,0.046949,0.208105,-0.185548,0.001031,0.098816,-0.552904,-0.073288,0.023307,6.14,0
1,158.0,-0.600816,0.922455,-0.135952,-1.259915,2.439824,3.330205,0.087107,0.949659,-0.327932,...,-0.311489,-0.927233,-0.045268,0.979587,0.016423,0.091043,0.371280,0.180187,1.79,0
2,190.0,-1.505779,-0.215325,1.991294,-1.631493,-0.635965,0.228414,-0.034266,0.042696,-0.235440,...,-0.101938,0.148146,-0.450115,-0.396558,0.554224,-0.340493,-0.335618,-0.413379,82.29,0
3,265.0,-0.491003,0.906953,1.645423,-0.083531,-0.195560,-0.710165,0.559119,0.116340,-0.538190,...,-0.168067,-0.517387,0.018650,0.491652,-0.277795,0.043841,0.253372,0.111749,9.03,0
4,459.0,-0.528218,0.981232,1.652988,-0.150715,0.137048,-0.203980,0.583370,0.165246,-0.781696,...,-0.123569,-0.272731,0.023887,0.223936,-0.297777,0.068300,0.281018,0.111557,6.99,0


In [19]:
nm_dataset = pd.concat([nm_features, nm_labels], axis=1)
nm_dataset.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,76866.0,1.257227,0.408096,0.319266,0.685011,-0.302937,-1.052552,0.136991,-0.245655,-0.079491,...,-0.279430,-0.770134,0.119789,0.360130,0.241921,0.092261,-0.018228,0.032361,1.78,0
1,76868.0,1.254775,0.375340,0.311959,0.691077,-0.343494,-1.065252,0.110222,-0.222721,-0.017278,...,-0.283275,-0.803506,0.124643,0.354395,0.226835,0.093208,-0.021100,0.031686,2.69,0
2,76870.0,-1.050697,1.500753,0.846326,-0.282956,0.059866,-0.561182,0.484517,0.181547,0.081544,...,-0.335560,-0.780232,-0.027029,-0.090180,-0.022896,0.091830,0.547241,0.278780,1.29,0
3,76866.0,1.245537,0.419648,0.312943,1.111459,-0.205612,-0.950912,0.267398,-0.274685,-0.127708,...,0.035687,0.186705,-0.107300,0.417887,0.712330,-0.330843,0.018576,0.021871,3.99,0
4,76870.0,1.261137,0.304662,0.527018,0.612765,-0.411488,-0.972770,0.112838,-0.248052,-0.068454,...,-0.249081,-0.683649,0.126936,0.395307,0.243448,0.092344,-0.024149,0.020589,1.79,0


In [20]:
sm_dataset['Class'].value_counts(), sm_dataset.shape, nm_dataset.Class.value_counts(), nm_dataset.shape

(0    283253
 1    283253
 Name: Class, dtype: int64,
 (566506, 31),
 0    473
 1    473
 Name: Class, dtype: int64,
 (946, 31))

import back to pyspark

In [21]:
sm_dataset = spark.createDataFrame(sm_dataset)
nm_dataset = spark.createDataFrame(nm_dataset)

/home/hanlinn/miniconda3/envs/deep-learning/lib/python3.9/site-packages/pyspark/sql/pandas/conversion.py:371: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


Scale the dataset

In [22]:
sm_dataset.count(), nm_dataset.count()

23/04/17 11:56:03 WARN TaskSetManager: Stage 38 contains a task of very large size (12657 KiB). The maximum recommended task size is 1000 KiB.


(566506, 946)

In [23]:
feature.columns, label.columns

(['Time',
  'V1',
  'V2',
  'V3',
  'V4',
  'V5',
  'V6',
  'V7',
  'V8',
  'V9',
  'V10',
  'V11',
  'V12',
  'V13',
  'V14',
  'V15',
  'V16',
  'V17',
  'V18',
  'V19',
  'V20',
  'V21',
  'V22',
  'V23',
  'V24',
  'V25',
  'V26',
  'V27',
  'V28',
  'Amount'],
 ['Class'])

Near Miss scale

In [24]:
# # select the columns to be scaled
# cols_to_scale = ['feature1', 'feature2', 'feature3']

# assemble the selected columns into a vector
assembler = VectorAssembler(inputCols=feature.columns, outputCol='features')
nm_assembled_df = assembler.transform(nm_dataset)

# standardize the features
scaler = StandardScaler(inputCol='features', outputCol='scaledFeatures', withMean=True, withStd=True)
nm_scaled_df = scaler.fit(nm_assembled_df).transform(nm_assembled_df)

In [25]:
nm_scaled_df.head(5), nm_scaled_df.count()

([Row(Time=76866.0, V1=1.25722726444768, V2=0.408096200805677, V3=0.319265682875671, V4=0.685010883570425, V5=-0.302936786959417, V6=-1.05255227554354, V7=0.136990827900143, V8=-0.245654767842237, V9=-0.079490728590699, V10=-0.304452607275493, V11=0.0556384574763668, V12=0.627844196900779, V13=0.968874464147768, V14=-0.435499318033619, V15=1.01292648524392, V16=0.44528377280571, V17=-0.0496221928840565, V18=-0.388479970247941, V19=-0.201394734414342, V20=-0.0330528283242017, V21=-0.279430006757678, V22=-0.770134097492737, V23=0.119788585978771, V24=0.360129780489771, V25=0.241920630942117, V26=0.0922609639515654, V27=-0.0182280075007381, V28=0.0323610755522783, Amount=1.78, Class=0, features=DenseVector([76866.0, 1.2572, 0.4081, 0.3193, 0.685, -0.3029, -1.0526, 0.137, -0.2457, -0.0795, -0.3045, 0.0556, 0.6278, 0.9689, -0.4355, 1.0129, 0.4453, -0.0496, -0.3885, -0.2014, -0.0331, -0.2794, -0.7701, 0.1198, 0.3601, 0.2419, 0.0923, -0.0182, 0.0324, 1.78]), scaledFeatures=DenseVector([-0.243

SMOTE scale

In [26]:
# # select the columns to be scaled
# cols_to_scale = ['feature1', 'feature2', 'feature3']

# assemble the selected columns into a vector
assembler = VectorAssembler(inputCols=feature.columns, outputCol='features')
sm_assembled_df = assembler.transform(sm_dataset)

# standardize the features
scaler = StandardScaler(inputCol='features', outputCol='scaledFeatures', withMean=True, withStd=True)
sm_scaled_df = scaler.fit(sm_assembled_df).transform(sm_assembled_df)

23/04/17 11:56:06 WARN TaskSetManager: Stage 51 contains a task of very large size (12657 KiB). The maximum recommended task size is 1000 KiB.


In [27]:
sm_scaled_df.head(5), sm_scaled_df.count()

23/04/17 11:56:08 WARN TaskSetManager: Stage 54 contains a task of very large size (12657 KiB). The maximum recommended task size is 1000 KiB.
23/04/17 11:56:08 WARN TaskSetManager: Stage 55 contains a task of very large size (12657 KiB). The maximum recommended task size is 1000 KiB.


([Row(Time=26.0, V1=-0.529912284186556, V2=0.873891581460326, V3=1.34724732930113, V4=0.145456676582257, V5=0.414208858362661, V6=0.10022309405219, V7=0.711206082959649, V8=0.1760659570625, V9=-0.286716934699997, V10=-0.484687683196852, V11=0.872489590125871, V12=0.851635859904339, V13=-0.571745302934562, V14=0.100974273045751, V15=-1.51977183258512, V16=-0.284375978261788, V17=-0.310523584869201, V18=-0.404247868800905, V19=-0.823373523914155, V20=-0.290347610865436, V21=0.0469490671140629, V22=0.208104855076299, V23=-0.185548346773547, V24=0.00103065983293288, V25=0.0988157011025622, V26=-0.552903603040518, V27=-0.0732880835681738, V28=0.0233070451077205, Amount=6.14, Class=0, features=DenseVector([26.0, -0.5299, 0.8739, 1.3472, 0.1455, 0.4142, 0.1002, 0.7112, 0.1761, -0.2867, -0.4847, 0.8725, 0.8516, -0.5717, 0.101, -1.5198, -0.2844, -0.3105, -0.4042, -0.8234, -0.2903, 0.0469, 0.2081, -0.1855, 0.001, 0.0988, -0.5529, -0.0733, 0.0233, 6.14]), scaledFeatures=DenseVector([-1.8035, 0.33

other way of testing

In [28]:
# feature_cols = df.drop('Class').columns
# label_col = df.select('Class').columns

In [29]:
# assembler = VectorAssembler(inputCols = feature_cols, outputCol='features')
# output = assembler.transform(df)

In [30]:
# output = assembler.transform(df)

In [31]:
# finalised_data = output.select('features', 'Class')
# finalised_data.show()

In [32]:
# # split the data into training and test sets
# train_data, test_data = finalised_data.randomSplit([0.8, 0.2], seed=42)

# # create a LogisticRegression object
# from pyspark.ml.classification import LogisticRegression
# lr = LogisticRegression(labelCol='Class',maxIter=10, regParam=0.01)

# # fit the model on the training data
# model = lr.fit(train_data)

# # make predictions on the test data
# predictions = model.transform(test_data)



Pyspark logistic Regression

In [33]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

smote + logistic regression

In [34]:
sm_scaled_df.columns

['Time',
 'V1',
 'V2',
 'V3',
 'V4',
 'V5',
 'V6',
 'V7',
 'V8',
 'V9',
 'V10',
 'V11',
 'V12',
 'V13',
 'V14',
 'V15',
 'V16',
 'V17',
 'V18',
 'V19',
 'V20',
 'V21',
 'V22',
 'V23',
 'V24',
 'V25',
 'V26',
 'V27',
 'V28',
 'Amount',
 'Class',
 'features',
 'scaledFeatures']

In [35]:
final_sm_data = sm_scaled_df.select('scaledFeatures','Class')

In [36]:
train_data, test_data = final_sm_data.randomSplit([0.8, 0.2], seed=42)

In [37]:
# create the logistic regression model
lr = LogisticRegression(featuresCol='scaledFeatures', labelCol='Class', maxIter=10)

# train the model
lrModel = lr.fit(train_data)

# make predictions on the test set
predictions = lrModel.transform(test_data)

23/04/17 11:56:10 WARN TaskSetManager: Stage 58 contains a task of very large size (12657 KiB). The maximum recommended task size is 1000 KiB.
23/04/17 11:56:13 WARN TaskSetManager: Stage 60 contains a task of very large size (12657 KiB). The maximum recommended task size is 1000 KiB.
23/04/17 11:56:16 WARN TaskSetManager: Stage 62 contains a task of very large size (12657 KiB). The maximum recommended task size is 1000 KiB.
23/04/17 11:56:16 WARN TaskSetManager: Stage 64 contains a task of very large size (12657 KiB). The maximum recommended task size is 1000 KiB.
23/04/17 11:56:17 WARN TaskSetManager: Stage 66 contains a task of very large size (12657 KiB). The maximum recommended task size is 1000 KiB.
23/04/17 11:56:17 WARN TaskSetManager: Stage 68 contains a task of very large size (12657 KiB). The maximum recommended task size is 1000 KiB.
23/04/17 11:56:17 WARN TaskSetManager: Stage 70 contains a task of very large size (12657 KiB). The maximum recommended task size is 1000 KiB.

In [41]:
# evaluate the model using F1 score
evaluator = MulticlassClassificationEvaluator(predictionCol='prediction', labelCol='Class', metricName='f1')
f1_score = evaluator.evaluate(predictions)
print(f'F1 score: {f1_score}')


23/04/17 12:08:19 WARN TaskSetManager: Stage 84 contains a task of very large size (12657 KiB). The maximum recommended task size is 1000 KiB.


F1 score: 0.9684560610855982


In [42]:
final_nm_data = nm_scaled_df.select('scaledFeatures','Class')

In [43]:
train_data, test_data = final_nm_data.randomSplit([0.8, 0.2], seed=42)

In [44]:
# create the logistic regression model
lr = LogisticRegression(featuresCol='scaledFeatures', labelCol='Class', maxIter=10)

# train the model
lrModel = lr.fit(train_data)

# make predictions on the test set
predictions = lrModel.transform(test_data)

In [45]:
# evaluate the model using F1 score
evaluator = MulticlassClassificationEvaluator(predictionCol='prediction', labelCol='Class', metricName='f1')
f1_score = evaluator.evaluate(predictions)
print(f'F1 score: {f1_score}')


F1 score: 0.9239011739011738
